In [2]:
import os
os.chdir('../../')

In [3]:
import data
import preprocess_utils.session2vec as sess2vec
import utils.sparsedf as sparsedf
from utils.df import MinMaxScaler
from utils.dataset import SequenceDatasetForClassification, SequenceDatasetForRegression
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

Using TensorFlow backend.
/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
from recommenders.recommender_base import RecommenderBase
from recommenders.recurrent.RNNClassificationRecommender import RNNClassificationRecommender

from numpy.linalg import norm as L2Norm
from sklearn.metrics import classification_report

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, GRU, Embedding
from keras.callbacks import EarlyStopping

In [5]:
mode = 'small'

## Build/load the model

In [7]:
dataset = SequenceDatasetForClassification(f'dataset/preprocessed/cluster_recurrent/{mode}/dataset_classification_p6')
#dataset = SequenceDatasetForClassification(f'dataset/preprocessed/cluster_up_to_len6/{mode}/dataset_classification')
#dataset = SequenceDatasetForClassification(f'dataset/preprocessed/cluster_over_len6/{mode}/dataset_classification')

In [8]:
weights = dataset.get_class_weights()

Y_train: (37664, 25)
{0: 0.10620796616143814, 1: 0.3912126720332381, 2: 0.5512477131357483, 3: 0.7377864838393732, 4: 0.847334083239595, 5: 1.0297744360902255, 6: 1.223850528025995, 7: 1.3847058823529412, 8: 1.5420266120777892, 9: 1.735668202764977, 10: 1.799952210274791, 11: 1.9290140845070423, 12: 2.1993576642335766, 13: 2.5797260273972604, 14: 2.7145225225225227, 15: 3.055902636916836, 16: 3.068350305498982, 17: 3.6042105263157893, 18: 3.275130434782609, 19: 3.8530946291560104, 20: 3.954225721784777, 21: 4.184888888888889, 22: 4.593170731707317, 23: 4.782730158730159, 24: 4.082818428184281}


In [8]:
#model = RecurrentRecommender(dataset, loss='categorical_crossentropy', cell_type='gru', num_units=50, num_layers=2)
model = RNNClassificationRecommender(dataset, input_shape=(dataset.rows_per_sample, 168),
                                     cell_type='gru',
                                     num_recurrent_units=64,
                                     num_recurrent_layers=2, num_dense_layers=2,
                                     optimizer=keras.optimizers.Adam(lr=1e-2),
                                     class_weights=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 6, 64)             44736     
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1625      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
Total params: 75,289
Trainable params: 75,289
Non-trainable params: 0
_________________________________________________________________
None



In [12]:
x,y = dataset.load_Xtrain(), dataset.load_Ytrain()

In [9]:
x.shape

(37664, 6, 42)

In [13]:
x[0]

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [14]:
y[0]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

## Train the model

In [12]:
#model.fit(X_train, Y_train, epochs=10, validation_split=0.1)
model.fit(epochs=3)

X_train: (37664, 6, 168)
Train on 32014 samples, validate on 5650 samples
Epoch 1/3
32014/32014 [==============================] - 26s 801us/step - loss: 3.5650 - acc: 0.3919 - mrr: 0.5070 - val_loss: 2.0151 - val_acc: 0.4846 - val_mrr: 0.6023
Epoch 2/3
32014/32014 [==============================] - 19s 604us/step - loss: 3.4691 - acc: 0.4238 - mrr: 0.5350 - val_loss: 1.9940 - val_acc: 0.5009 - val_mrr: 0.6115
Epoch 3/3
32014/32014 [==============================] - 19s 595us/step - loss: 3.4104 - acc: 0.4345 - mrr: 0.5440 - val_loss: 2.0032 - val_acc: 0.4988 - val_mrr: 0.6108


In [9]:
#model.model.save('gru_permutation_importance.h5')
model.load('gru_permutation_importance.h5')

In [ ]:
model.load('gru_primo_tentativo_dopo_preprocessing.h5')

## Load saved model

In [6]:
model = RNNClassificationRecommender(dataset, cell_type='gru', num_recurrent_units=64, num_recurrent_layers=2,
                                     num_dense_layers=2, input_shape=(dataset.rows_per_sample, 170),
                                     class_weights=[])
model.load('saved_models/rnn_GRU_2layers_64units_2dense_class_066392.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 12, 64)            45120     
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1625      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
Total params: 75,673
Trainable params: 75,673
Non-trainable params: 0
_________________________________________________________________
None



In [8]:
model.load('gru.h5')

NameError: name 'model' is not defined

## Use the model

In [7]:
#X_test_df = pd.read_csv(f'dataset/preprocessed/cluster_recurrent/{mode}/X_test.csv').set_index('orig_index')
#X_test_df = scale_dataframe(X_test_df, ['impression_price'])

target_indices = data.target_indices(mode, 'cluster_recurrent')
#target_indices = data.target_indices(mode, 'cluster_recurrent_up_to_len6')
#target_indices = data.target_indices(mode)
#X_test_df.head(3)
print(len(target_indices))
print(target_indices[:10])

153130
[54066 54071 54108 54184 54211 54343 54397 54606 54609 54906]


In [8]:
recomendations = model.recommend_batch(target_indices)
if mode != 'full':
    model.compute_MRR(recomendations)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Scaler loaded!
X_test: (153130, 12, 167)
caching df_full...


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
  0%|          | 1/153130 [00:00<5:14:51,  8.11it/s]

Done!


100%|██████████| 153130/153130 [00:56<00:00, 2711.77it/s]

prediction created !!!
Loading full train_df, it will take a while..



100%|██████████| 153130/153130 [00:00<00:00, 887880.03it/s]

Calculating MRR (hoping for a 0.99)
MRR: 0.6627973069423838


## Classification report

In [98]:
xtest, indices = dataset.load_Xtest()

# build y_true
def get_y_true(clickout_indices, mode):
    df = data.full_df().loc[clickout_indices]
    
    def add_label(row):
        impress = list(map(int, row['impressions'].split('|')))
        ref = row['reference']
        #if pd.isnull(ref):
        #    return np.NaN

        if ref in impress:
            return impress.index(ref)
        else:
            return 0
    
    df = df.astype({'reference':'int'})
    df['label'] = df.progress_apply(add_label, axis=1)
    return df['label']
 
yclasses = get_y_true(indices, mode)

#y_true = np.zeros((len(yclasses), 25))
#y_true[np.arange(y_true.shape[0]),yclasses] = 1

In [101]:
y_pred = model.model.predict_classes(xtest)

## rnn_GRU_2layers_64units_2dense_class_065193

In [103]:
print(classification_report(yclasses, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.93      0.69     58083
           1       0.55      0.31      0.40     15639
           2       0.56      0.32      0.41     10693
           3       0.51      0.33      0.40      8501
           4       0.53      0.31      0.39      7315
           5       0.54      0.32      0.40      5750
           6       0.51      0.31      0.39      4917
           7       0.53      0.32      0.40      4393
           8       0.55      0.32      0.41      3947
           9       0.53      0.32      0.40      3655
          10       0.55      0.30      0.39      3275
          11       0.53      0.31      0.39      2810
          12       0.55      0.31      0.40      2741
          13       0.57      0.30      0.40      2480
          14       0.56      0.32      0.41      2179
          15       0.58      0.31      0.40      2125
          16       0.59      0.33      0.42      2005
          17       0.60    

### Features:
ReferencePositionInLastClickoutImpressions,

GlobalClickoutPopularity,

ReferencePriceInLastClickout,

ReferencePricePositionInLastClickout,

SessionsImpressionsCount,

ClickoutVectorPrices,

InteractionDuration,

2x64 gru - 2 dense
#### 0.64470

### Features:
ReferencePositionInLastClickoutImpressions,

GlobalClickoutPopularity,

ReferencePriceInLastClickout,

SessionsImpressionsCount

2x64 gru - 2 dense
#### 0.64273


### Features
ReferencePositionInLastClickoutImpressions,

ReferencePriceInLastClickout,

ClickoutVectorPrices,
#### 0.62387


### Cluster len<=6

With 1 time distributed: 0.625722

Naive: 0.625908

## Export sub

In [7]:
import out

def create_sub(model, sub_suffix=''):
    mode = 'full'
    target_indices = data.target_indices(mode, 'cluster_recurrent')
    print('Recommending...')
    recomendations = model.recommend_batch(target_indices)
    out.create_sub(recomendations, submission_name=f'{model.name}_{sub_suffix}')
    print('Done!')

In [8]:
create_sub(model)

Recommending...


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


X_test: (253573, 12, 118)
caching df_full...


/Users/federico/Documents/Repository/recsys2019/recommenders/recurrent/RNNClassificationRecommender.py:50: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  predictions = pred_df.loc[target_indices]
/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
  0%|          | 1/253573 [00:00<7:15:37,  9.70it/s]

Done!


100%|██████████| 253573/253573 [01:32<00:00, 2741.40it/s]


prediction created !!!
Exporting the sub to submissions/rnn_GRU_2layers_64units_2dense_wgt_class__09-56-34.csv...

submission created in submissions folder in 00m 04s
Done!


In [11]:
import out
out.create_sub(recomendations, submission_name=f'{model.name}_066279')

Exporting the sub to submissions/rnn_GRU_2layers_64units_2dense_wgt_class_066279_18-35-41.csv...

submission created in submissions folder in 00m 04s


'submissions/rnn_GRU_2layers_64units_2dense_wgt_class_066279_18-35-41.csv'

## Scores

In [ ]:
xtest, indices = dataset.load_Xtest()

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [7]:
scores = model.get_scores_batch()

X_test: (153135, 6, 68)
caching df_full...


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
605it [00:00, 1896.59it/s]

Done!


153135it [00:33, 4583.94it/s]


In [10]:
model.name

'rnn_GRU_2layers_64units_2dense_class'

In [10]:
np.save('rnn_GRU_2layers_64units_2dense_wgt_class_055780_balanced', np.array(scores))

## Permutation importance

In [11]:
from eli5.sklearn.permutation_importance import PermutationImportance

In [12]:
def score_fn(model, X, y):
    print(X.shape, y.shape)
    
    target_indices = data.target_indices(model.mode, 'cluster_recurrent')
    
    _, indices = self.dataset.load_Xtest()
        
    # predict the references
    predictions = self.model.predict(X)

    # take only the target predictions
    pred_df = pd.DataFrame(predictions)
    pred_df['orig_index'] = indices
    pred_df = pred_df.set_index('orig_index')
    predictions = pred_df.loc[target_indices]
    del pred_df

    assert len(predictions) == len(target_indices)

    full_df = data.full_df()

    result_predictions = []
    for index in tqdm(target_indices):
        # get the impressions of the clickout to predict
        impr = list(map(int, full_df.loc[index]['impressions'].split('|')))
        # build a list of (impression, score)
        prediction_impressions_distances = [ (impr[j], predictions.at[index,j]) for j in range(len(impr)) ]
        # order the list based on scores (greater is better)
        prediction_impressions_distances.sort(key=lambda tup: tup[1], reverse=True)
        # get only the impressions ids
        ordered_impressions = list(map(lambda x: x[0], prediction_impressions_distances))
        # append the couple (index, reranked impressions)
        result_predictions.append( (index, ordered_impressions) )

    if model.mode == 'full':
        print('Invalid mode (full)!')
        return None
    
    return model.compute_MRR(result_predictions)

In [13]:
perm_importance = PermutationImportance(model, score_fn)

In [14]:
x, _ = dataset.load_Xtest()
print(x.shape)
y = np.zeros((x.shape[0],1))
print(y.shape)

X_test: (9246, 6, 168)
(9246, 6, 168)
(9246, 1)


In [16]:
perm_importance.fit(np.array([[None, None]]), y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').